In [1]:
import sys

import torch
import torch.nn.functional as fn
from torch import Tensor
from torch.nn import Module

sys.path.append("..")
from pfhedge.instruments import BrownianStock
from pfhedge.instruments import EuropeanOption
from pfhedge.nn import BlackScholes
from pfhedge.nn import Clamp
from pfhedge.nn import Hedger
from pfhedge.nn import MultiLayerPerceptron
from torch.nn import Module

In [5]:
class NoTransactionBandNet(Module):
    def __init__(self, derivative):
        super().__init__()

        self.delta = BlackScholes(derivative)
        self.mlp = MultiLayerPerceptron(out_features=2)
        self.clamp = Clamp()

    def inputs(self):
        return self.delta.inputs() + ["prev_hedge"]

    def forward(self, input: Tensor) -> Tensor:
        prev_hedge = input[..., [-1]]

        delta = self.delta(input[..., :-1])
        width = self.mlp(input[..., :-1])

        min = delta - fn.leaky_relu(width[..., [0]])
        max = delta + fn.leaky_relu(width[..., [1]])

        return self.clamp(prev_hedge, min=min, max=max)

In [ ]:
def to_numpy(tensor: torch.Tensor) -> np.array:
    return tensor.cpu().detach().numpy()

# Custom transaction cost function
def custom_transaction_cost(traded_volume):
    return torch.abs(traded_volume) ** (3/4)

inputs = ["log_moneyness", "time_to_maturity"]

underlier=BrownianStock(cost=1e-4)
# Define the derivative (e.g., European Put Option)
derivative = EuropeanOption(
    underlier, strike=100, maturity=30/252
)

# Initialize the hedger and integrate transaction cost
hedger = Hedger(
    model = BlackScholes(derivative),
    inputs=inputs,
    cost=custom_transaction_cost  # Custom transaction cost
)

custom_hedger = Hedger(
    model=NoTransactionBandNet(),
    inputs=inputs,
    cost=custom_transaction_cost,
)


TypeError: Hedger.__init__() got an unexpected keyword argument 'cost'

In [ ]:
# In each epoch, N_PATHS brownian motion time-series are generated.
N_PATHS = 50000
# How many times a model is updated in the experiment.
N_EPOCHS = 200

history = hedger.fit(derivative, n_epochs=N_EPOCHS, n_paths=N_PATHS, n_times=20)